In [ ]:
import torch
import torch.nn
import torch.nn.functional as F

#using GPU if available

device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)

cuda


In [ ]:
# function to read a text file and return all characters present in it

def readTextFile(path):

    with open(path, 'r', encoding = 'UTF-8') as f:

        text = f.read()

    characters = sorted(set(text))

    return text, characters


In [ ]:
text, characters = readTextFile('./src/wizardOfOz.txt')

# lambda functions to encode and decode the characters of the text

stringToNum = {ch: i for i, ch  in enumerate(characters)}

numToString = {i: ch for i,ch in enumerate(characters)}

encode = lambda s: [stringToNum[c] for c in s]

decode = lambda s: [numToString[n] for n in s]

# converting the text into a tensor

data = torch.tensor(encode(text))